In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
import pickle
import dask.dataframe as dd
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import subprocess
import random

/tmp/ipykernel_1786473/1276407396.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'dask'

In [2]:
## create the env testgin 

with open('splits_samples.pkl', 'rb') as file:
    # Load data from the file
    splits_samples = pickle.load(file)

## split0 

for i in range(100):
    train_samples = splits_samples[i][0]
    test_samples = splits_samples[i][1]
    
    clim_sites_during_exp = pd.read_csv('/carnegie/nobackup/scratch/tbellagio/grene/data/bioclimvars_experimental_sites_era5.csv')
    
    sites_af = pd.Series(train_samples).str.split('_').str[0].astype(int)
    
    sites_af.name = 'site'
    
    env = sites_af.reset_index().merge(clim_sites_during_exp).drop(['index'],axis=1)
    
    ## for now only work with bio1 and bio12 
    env = env[['site', 'bio1', 'bio12']]
    
    mean_bio1 = np.mean(env['bio1'])
    mean_bio12 = np.mean(env['bio12'])
    std_bio1 = np.std(env['bio1'])
    std_bio12 = np.std(env['bio12'])
    
    ## for the testing
    
    sites_af = pd.Series(test_samples).str.split('_').str[0].astype(int)
    sites_af.name = 'site'
    env = sites_af.reset_index().merge(clim_sites_during_exp).drop(['index'],axis=1)
    env = env[['site', 'bio1', 'bio12']]

    ##scale it
    
    env['bio1'] = (env['bio1'] - mean_bio1) / std_bio1 
    env['bio12'] = (env['bio12'] - mean_bio12) / std_bio12
    file = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_{i}/testing_environment_lea_full_genome.csv'
    print(file)
    env.to_csv(file)

/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_0/testing_environment_lea_full_genome.csv
/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_1/testing_environment_lea_full_genome.csv
/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_2/testing_environment_lea_full_genome.csv
/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_3/testing_environment_lea_full_genome.csv
/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_4/testing_environment_lea_full_genome.csv
/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_5/testing_environment_lea_full_genome.csv
/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_6/testing_environment_lea_full_genome.csv
/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_7/testing_environment_lea_full_genome.csv
/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave

In [27]:
## create the env testgin 

with open('splits_sites.pkl', 'rb') as file:
    # Load data from the file
    splits_sites = pickle.load(file)

In [29]:

for i in range(100):
## get the training and testing sites 
    train_samples = splits_sites[i][0]
    test_samples = splits_sites[i][1]
    ## get the climate data 
    clim_sites_during_exp = pd.read_csv('/carnegie/nobackup/scratch/tbellagio/grene/data/bioclimvars_experimental_sites_era5.csv')
    


    ## use th train samples convert it into mergeable object 
    sites_af = pd.Series(train_samples)
    
    sites_af.name = 'site'
    ## env trainign 
    env = sites_af.reset_index().merge(clim_sites_during_exp).drop(['index'],axis=1)

    ## for now only work with bio1 and bio12 
    env = env[['site', 'bio1', 'bio12']]
    
    mean_bio1 = np.mean(env['bio1'])
    mean_bio12 = np.mean(env['bio12'])
    std_bio1 = np.std(env['bio1'])
    std_bio12 = np.std(env['bio12'])

    ## standarize training dataset 
    env['bio1'] = (env['bio1'] - mean_bio1) / std_bio1 
    env['bio12'] = (env['bio12'] - mean_bio12) / std_bio12

    file_train = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_{i}/env_training_sites.csv'
    env.to_csv(file_train)


    
    ## for the testing
    
    sites_af = pd.Series(test_samples)
    sites_af.name = 'site'
    env = sites_af.reset_index().merge(clim_sites_during_exp).drop(['index'],axis=1)
    env = env[['site', 'bio1', 'bio12']]

    ##scale it
    
    env['bio1'] = (env['bio1'] - mean_bio1) / std_bio1 
    env['bio12'] = (env['bio12'] - mean_bio12) / std_bio12


    file_test = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_{i}/env_testing_sites.csv'


    env.to_csv(file_test)


In [33]:
pd.read_csv('/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_1/environment_lea_full_genome.csv').columns

Index(['bio1', 'bio12'], dtype='object')

In [31]:
pd.read_csv('/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_1/env_training_sites.csv')

,Unnamed: 0,site,bio1,bio12
0,0,1,-0.755693,0.074908
1,1,54,-0.349904,-0.010640
2,2,57,-0.389063,-0.099534
3,3,27,-1.549624,-0.477820
4,4,5,0.261600,-0.902266
5,5,9,-0.339976,1.499131
6,6,45,0.418993,-1.029567
7,7,23,-1.581744,0.381531
8,8,4,0.863758,-0.267216
9,9,53,-0.368552,-0.852456


In [32]:
pd.read_csv('/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_1/testing_environment_lea_full_genome.csv')

,Unnamed: 0,site,bio1,bio12
0,0,2,0.403359,0.529108
1,1,2,0.403359,0.529108
2,2,2,0.403359,0.529108
3,3,2,0.403359,0.529108
4,4,2,0.403359,0.529108
5,5,2,0.403359,0.529108
6,6,2,0.403359,0.529108
7,7,11,-0.031177,1.003208
8,8,11,-0.031177,1.003208
9,9,11,-0.031177,1.003208
